In [2]:
import numpy as np
import pandas as pd
import ast

In [31]:
bad_words = pd.read_csv('data/bad_word.txt',header=None)
bad_words_set = set(bad_words)

In [14]:
pos_tags_dataset = pd.read_csv('data/pos_tags_dataset.csv')
pos_tags=[]
# for i in range(len(pos_tags_dataset)):
#     pos_tags.append(ast.literal_eval(pos_tags_dataset['pos_tag'][i]))

In [19]:
import re
labels= []
word_indices=[]
for i,sentence in enumerate(pos_tags_dataset['cleaned_comments']):
    #capture bw indices for each sentence
    try:
        for j,word in enumerate(sentence.split()):
            if word.lower() in bad_words_set:
                pos_tags[i][j] = 'BW'
    except:
        continue

In [23]:
pos_tags[238]

['WP',
 'IN',
 'DT',
 'BW',
 'VBP',
 'PRP',
 'IN',
 'IN',
 'DT',
 'VBZ',
 'NN',
 'TO',
 'VB',
 'IN',
 'NNP',
 'PRP',
 'DT',
 'VBP',
 'NN',
 'NN']

In [27]:
pos_tags_dataset['pos_tag'] = pos_tags

In [28]:
pos_tags_dataset.to_csv('data/pos_tags_dataset_bw.csv',index=False)